In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
#import lightning.pytorch as pl
import glob

In [ ]:
## First, we slice the train images into 31 x 31 pixels with the ground truth in the middle

In [43]:
# def ndigit(n, x):
#     x = str(x)
#     while(len(x) < n):
#         x = "0" + x
#     return x

In [64]:
# def load_data(res, files = 20, test = False):
#     j = 0
#     if (test == True):
#         path = "02"
#     else:
#         path = "train"
#     res = int((res-1)/2)
    
#     for n in range(files):
#         image = np.load(f"images_{path}/images/image_{ndigit(3, n)}.npy")
#         masks = np.load(f"masks_{path}/masks/mask_{ndigit(3, n)}.npy")
#         masks = np.reshape(masks, (1024,1024,1))
#         ground_truths_pos = np.array(np.where(masks != 0)).T
#         for i in ground_truths_pos:
#             train = (image[:, i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1], masks[i[0], i[1], 0])
#             np.save(f"images_{path}/train/train_{ndigit(5, j)}.npy", train)
#             j = j + 1

In [65]:
#load_data(31)

## Then, we load the data and have a look

In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(), # Converts an image to a Tensor
     transforms.ConvertImageDtype(torch.float),
     transforms.Normalize((0.5)*10, # Mean for RGB
                          (0.5)*10) # Std for RGB
     ]) 

batch_size = 128

In [5]:
directory = 'images_train/train'
file_paths = glob.glob(directory + '/*.npy')
trainset = [np.load(file_path, allow_pickle=True) for file_path in file_paths]

for pic in trainset:
    np.place(pic[0], pic[0] > 3000, 1)


In [11]:
for pic in trainset:
        #hier gibt es wohl bilder, die ein anderes shape haben, EORR \n",
        pic[0] = np.reshape(pic[0], (31,31,10))
        print(pic[0].shape)
        pic[0] = transform(pic[0])

# Split train set into new train set and validation set
val_size = round(0.2 * len(trainset))
train_size = round(0.8 * len(trainset))
trainset, valset = random_split(trainset, [train_size, val_size])

trainloader = DataLoader(trainset, batch_size=batch_size,
                         shuffle=True, num_workers=0, transform=transform)
validloader = DataLoader(trainset, batch_size=batch_size,
                         shuffle=True, num_workers=0, transform=transform)

(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)
(31, 31, 10)


ValueError: cannot reshape array of size 7130 into shape (31,31,10)

In [ ]:
testset = ...
testloader = DataLoader(testset, batch_size=batch_size,
                        shuffle=False, num_workers=0)

In [17]:
f, axarr = plt.subplots(1,10, figsize=(12, 12))
for i in range(10):
    X,y = trainset[i]
    X,y = X.transpose(0,-1).transpose(0,1) * 0.5 + 0.5, y
    axarr[i].imshow(X)
    axarr[i].axis('off')
    axarr[i].set_title(f'{y}', fontsize='small')

IndexError: list index out of range

: 

: 

## Next, we define the model and train it

In [ ]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

def forward(self, X):
    return self.layers(X)

def predict(self, X):
    with torch.no_grad():
        y_hat = self(X).argmax(1)
    if self.classes is not None:
        y_hat = [self.classes[i] for i in y_hat]
    return y_hat

def training_step(self, batch, batch_idx, log_prefix='train'): # New !
    X, y = batch # Tuple with (X,y) in our case
    y_hat = self(X)
    loss = F.cross_entropy(y_hat, y)
    self.log(f"{log_prefix}_loss", loss.item(), on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

def validation_step(self, batch, batch_idx): # New!
    with torch.no_grad():
        return self.training_step(batch, batch_idx, log_prefix='valid')

def configure_optimizers(self):
    # Adam with Weight Decay (Most commonly used)
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

    # Simplest scheduler is ReduceLROnPlateau. This scheduler reduces the learning rate by 0.1
    # if the val_loss has not decreased within the last 10 epochs.
    scheduler = {
        # REQUIRED: The scheduler instance
        "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
        # The unit of the scheduler's step size, could also be 'step'.
        # 'epoch' updates the scheduler on epoch end whereas 'step'
        # updates it after a optimizer update.
        "interval": "epoch",
        # How many epochs/steps should pass between calls to
        # `scheduler.step()`. 1 corresponds to updating the learning
        # rate after every epoch/step.
        "frequency": 1,
        # Metric to to monitor for schedulers like `ReduceLROnPlateau`
        "monitor": "val_loss",
        # If set to `True`, will enforce that the value specified 'monitor'
        # is available when the scheduler is updated, thus stopping
        # training if not found. If set to `False`, it will only produce a warning
        "strict": True,
        # If using the `LearningRateMonitor` callback to monitor the
        # learning rate progress, this keyword can be used to specify
        # a custom logged name
        "name": None,
    }
    return {"optimizer": optimizer, 'lr-scheduler': scheduler}

## Fit function deleted

In [ ]:
# Implement the original ResNet Layer.
# Find details on ResNet: https://arxiv.org/abs/1512.03385
# Find details on Batch Normalization: https://arxiv.org/abs/1502.03167
class MyResLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)
            
        self.res_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel, **kwargs),
            nn.BatchNorm2d(out_channels),
            #nn.ReLU(),
        )
        
    def forward(self, x):
        x_res = self.res_block(x)
        x = self.proj_out(x)
        return  F.relu(x + x_res) #x + x_res

tree_model = MyCNNModel(
    MyResLayer(3, 8, (3,3), padding='same'),
    MyResLayer(8, 8, (3,3), padding='same'),
    nn.MaxPool2d(2, 2),
    MyResLayer(8, 16, (3,3), padding='same'),
    MyResLayer(16, 16, (3,3), padding='same'),
    nn.MaxPool2d(2, 2),
    MyResLayer(16, 32, (3,3), padding='same'),
    MyResLayer(32, 32, (3,3), padding='same'),
    nn.MaxPool2d(2, 2),
    MyResLayer(32, 32, (3,3), padding='same'),
    MyResLayer(32, 32, (3,3), padding='same'),
    nn.MaxPool2d(2, 2),
    MyResLayer(32, 32, (3,3), padding='same'),
    MyResLayer(32, 32, (3,3), padding='same'),
    nn.MaxPool2d(2, 2),
    MyResLayer(32, 32, (3,3), padding='same'),
    MyResLayer(32, 32, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(32, ...),
    lr=0.01
)

In [ ]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='32-true', max_epochs=1,
                      callbacks=[RichProgressBar(refresh_rate=50),
                                 RichModelSummary(3),
                                ])

In [ ]:
trainer1.fit(tree_model, trainloader, testloader)

## Now, we can apply it

In [ ]:
f, axarr = plt.subplots(1,10, figsize=(12, 12))
for i in range(10):
    X,y = testset[i]
    y_hat = tree_model.predict(X.unsqueeze(0))[0]
    X,y = X.transpose(0,-1).transpose(0,1) * 0.5 + 0.5, testset.classes[y]
    axarr[i].imshow(X)
    axarr[i].axis('off')
    axarr[i].set_title(f'{y} - {y_hat}', fontsize='small')